In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [3]:
def parse_html(url: str):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

In [4]:
def get_data(url: str):
    df = pd.DataFrame()
    page = parse_html(url)

    while page.find("a", class_="nfl-o-table-pagination__next") or page.find(
        "table", class_="d3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable"
    ):  # this condition can be improved
        data = pd.read_html(url)[0]
        df = pd.concat([df, data])
        if not page.find("a", class_="nfl-o-table-pagination__next"):
            break
        url = url + page.find("a", class_="nfl-o-table-pagination__next")["href"]
        page = parse_html(url)

    df.index = df.reset_index().index
    print("done!")
    return df

In [8]:
stat_names = [["rushing","rushingattempts","Att",150],["passing","passingattempts","Att",200],["receiving","receivingreceptions","Rec",70]]
stat_dfs = []

for names in stat_names:

    beginning_url = "https://www.nfl.com/stats/player-stats/category/" + names[0] + "/"
    end_url = "/post/all/" + names[1] + "/desc"

    df = pd.DataFrame()
    for year in range(1994, 2021, 1):
        url = beginning_url + str(year) + end_url
        data = get_data(url)
        data = data.drop(data[data[names[2]] < names[3]].index)
        if names[0] == "receiving":
            data = data.drop(['Rec YAC/R', 'Tgts'], axis=1)
        data["Year"] = year
        df = pd.concat([df, data], ignore_index=True)
    
    df["pro_bowl"] = 0
    stat_dfs.append(df)

stat_dfs

done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!
done!


[               Player  Rush Yds  Att  TD  20+  40+  Lng  Rush 1st  Rush 1st%  \
 0        Emmitt Smith      1484  368  21    7    1   46       101       27.4   
 1       Natrone Means      1350  343  12    4    0   25        79       23.0   
 2        Chris Warren      1545  333   9   13    1   41        71       21.3   
 3       Barry Sanders      1883  331   7   18    7   85        72       21.8   
 4      Rodney Hampton      1075  327   6    2    0   27        58       17.7   
 ..                ...       ...  ...  ..  ...  ...  ...       ...        ...   
 669        Mike Davis       642  165   6    2    0   25        38       23.0   
 670     Miles Sanders       867  164   6    4    3   82        39       23.8   
 671     Lamar Jackson      1005  159   7   10    2   50        56       35.2   
 672   Adrian Peterson       604  156   7    4    0   38        34       21.8   
 673  Devin Singletary       687  156   2    3    1   51        40       25.6   
 
      Rush FUM  Year  pro_

In [9]:
pbdf = pd.DataFrame()
for year in range(1994, 2021, 1):
    pbdata = pd.read_html("https://www.pro-football-reference.com/years/" + str(year) + "/probowl.htm")
    pbdata = pd.DataFrame(np.squeeze(pbdata))
    names = [player.rstrip("%") for player in pbdata[1].to_list()]
    pbdata[1] = names
    pbdata = pbdata.drop(pbdata[((pbdata[0] != 'QB') & (pbdata[0] != 'RB') & (pbdata[0] != 'WR')) | (pbdata[1].str.find('+') != -1)].index)
    pbdata = pbdata.drop(pbdata.iloc[:,2:23], axis=1)
    pbdata["Year"] = year
    pbdf = pd.concat([pbdf,pbdata], ignore_index=True)

pbdf

,0,1,Year
0,WR,Cris Carter,1994
1,RB,Barry Sanders,1994
2,RB,Emmitt Smith,1994
3,WR,Jerry Rice,1994
4,QB,Steve Young,1994
...,...,...,...
644,WR,DeAndre Hopkins,2020
645,WR,Justin Jefferson,2020
646,WR,D.K. Metcalf,2020
647,WR,A.J. Brown,2020


In [10]:
posdict = dict()
posdict['RB'], posdict['QB'], posdict['WR'] = 0, 1, 2



for index, row in pbdf.iterrows():
    current_df = stat_dfs[posdict.get(row[0])]
    mask_df = current_df.isin({"Player" : [row[1]], "Year" : [row['Year']]})
    result = mask_df.index[((mask_df['Year'] == True) & (mask_df['Player'] == True))].tolist()
    stat_dfs[posdict.get(row[0])]['pro_bowl'][result] = 1

for idx, df in enumerate(stat_dfs):
    name = "nfl_" + stat_names[idx][0] + ".csv"
    df.to_csv(name, index=False)

C:\Users\colef\AppData\Local\Temp\ipykernel_17336\3355230860.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  stat_dfs[posdict.get(row[0])]['pro_bowl'][result] = 1
C:\Users\colef\AppData\Local\Temp\ipykernel_17336\3355230860.py:10: Setti

In [13]:
stat_dfs[1]

,Player,Pass Yds,Yds/Att,Att,Cmp,Cmp %,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY,Year,pro_bowl
0,Drew Bledsoe,4555,6.6,691,400,57.9,25,27,73.6,242,35.0,52,3,62,22,139,1994,1
1,Dan Marino,4453,7.2,615,385,62.6,30,17,89.2,214,34.8,55,7,64,18,113,1994,1
2,Warren Moon,4264,7.1,601,371,61.7,18,19,79.9,201,33.4,54,8,65,29,235,1994,1
3,Brett Favre,3882,6.7,582,363,62.4,33,14,90.7,200,34.4,44,4,49,31,188,1994,0
4,Jim Everett,3855,7.1,540,346,64.1,22,18,84.9,193,35.7,39,5,78,21,164,1994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,Carson Wentz,2620,6.0,437,251,57.4,16,15,72.8,130,29.8,27,6,59,50,326,2020,0
671,Joe Burrow,2688,6.6,404,264,65.4,13,5,89.8,150,37.1,23,3,67,32,231,2020,0
672,Drew Brees,2942,7.5,390,275,70.5,24,6,106.4,149,38.2,31,8,52,13,89,2020,0
673,Lamar Jackson,2757,7.3,376,242,64.4,26,9,99.4,138,36.7,37,4,47,29,160,2020,0
